In [1]:
import os
import numpy as np
import struct
from typing import Callable, Dict, Optional, Tuple
import multiprocessing
from concurrent.futures import ThreadPoolExecutor
import time
import torch
from torchvision import transforms
from matplotlib import pyplot as plt
import imageio

In [2]:
dvs_path = "/home/zhang/S2/RP/DataSet/DVS_Gesture_dataset/DvsGesture"
testData_path = "/home/zhang/S2/RP/DataSet/TestData"
events_dvs_split_path = "/home/zhang/S2/RP/DataSet/DVS_npz_events"
frames_dvs_split_path = "/home/zhang/S2/RP/DataSet/DVS_npz_frames"
visualization_saved_path= "/home/zhang/S2/RP/DataSet/Visualization_saved"

In [3]:
def load_aedat_v3(file_name: str) -> Dict:

    with open(file_name, 'rb') as bin_f:
        # skip ascii header
        line = bin_f.readline()
        while line.startswith(b'#'):
            if line == b'#!END-HEADER\r\n':
                break
            else:
                line = bin_f.readline()

        txyp = {
            't': [],
            'x': [],
            'y': [],
            'p': []
        }
        while True:
            header = bin_f.read(28)
            if not header or len(header) == 0:
                break

            # read header
            e_type = struct.unpack('H', header[0:2])[0]
            e_source = struct.unpack('H', header[2:4])[0]
            e_size = struct.unpack('I', header[4:8])[0]
            e_offset = struct.unpack('I', header[8:12])[0]
            e_tsoverflow = struct.unpack('I', header[12:16])[0]
            e_capacity = struct.unpack('I', header[16:20])[0]
            e_number = struct.unpack('I', header[20:24])[0]
            e_valid = struct.unpack('I', header[24:28])[0]

            data_length = e_capacity * e_size
            data = bin_f.read(data_length)
            counter = 0

            if e_type == 1:
                while data[counter:counter + e_size]:
                    aer_data = struct.unpack('I', data[counter:counter + 4])[0]
                    timestamp = struct.unpack('I', data[counter + 4:counter + 8])[0] | e_tsoverflow << 31
                    x = (aer_data >> 17) & 0x00007FFF
                    y = (aer_data >> 2) & 0x00007FFF
                    pol = (aer_data >> 1) & 0x00000001
                    counter = counter + e_size
                    txyp['x'].append(x)
                    txyp['y'].append(y)
                    txyp['t'].append(timestamp)
                    txyp['p'].append(pol)
            else:
                # non-polarity event packet, not implemented
                pass
        txyp['x'] = np.asarray(txyp['x'])
        txyp['y'] = np.asarray(txyp['y'])
        txyp['t'] = np.asarray(txyp['t'])
        txyp['p'] = np.asarray(txyp['p'])
        return txyp

In [4]:
#function test
d = load_aedat_v3(os.path.join(testData_path,"user01_fluorescent_led.aedat"))
for i in d.keys():
    print(i,d[i])

t [ 72406439  72406447  72406503 ... 192326082 192326339 192326718]
x [46 49 45 ... 46 89 64]
y [79 96 69 ... 29 14 46]
p [1 1 1 ... 1 0 1]


In [4]:
def split_aedat_files_to_npz(fname: str, aedat_file: str, csv_file: str, output_dir: str):
    events = load_aedat_v3(aedat_file)
    print(f'Start to split [{aedat_file}] to samples.')
    # read csv file and get time stamp and label of each sample
    # then split the origin data to samples
    csv_data = np.loadtxt(csv_file, dtype=np.uint32, delimiter=',', skiprows=1)

    # Note that there are some files that many samples have the same label, e.g., user26_fluorescent_labels.csv
    file_num = [0] * 11

    for i in range(csv_data.shape[0]):
        # the label of DVS128 Gesture is 1, 2, ..., 11. We set 0 as the first label, rather than 1
        label = csv_data[i][0] - 1
        t_start = csv_data[i][1]
        t_end = csv_data[i][2]
        mask = np.logical_and(events['t'] >= t_start, events['t'] < t_end)
        output_file_name = os.path.join(output_dir, str(label), f'{fname}_{file_num[label]}.npz')
        np.savez(output_file_name,
                    t=events['t'][mask],
                    x=events['x'][mask],
                    y=events['y'][mask],
                    p=events['p'][mask]
                    )
        print(f'[{output_file_name}] saved.')
        file_num[label] += 1

In [8]:
#function test
split_aedat_files_to_npz("user26_fluorescent",os.path.join(testData_path,"user26_fluorescent.aedat",),os.path.join(testData_path,"user26_fluorescent_labels.csv"),os.path.join(testData_path,"outputTest"))

Start to split [/home/zhang/S2/RP/DataSet/TestData/user26_fluorescent.aedat] to samples.
[/home/zhang/S2/RP/DataSet/TestData/outputTest/0/user26_fluorescent_0.npz] saved.
[/home/zhang/S2/RP/DataSet/TestData/outputTest/1/user26_fluorescent_0.npz] saved.
[/home/zhang/S2/RP/DataSet/TestData/outputTest/2/user26_fluorescent_0.npz] saved.
[/home/zhang/S2/RP/DataSet/TestData/outputTest/3/user26_fluorescent_0.npz] saved.
[/home/zhang/S2/RP/DataSet/TestData/outputTest/4/user26_fluorescent_0.npz] saved.
[/home/zhang/S2/RP/DataSet/TestData/outputTest/5/user26_fluorescent_0.npz] saved.
[/home/zhang/S2/RP/DataSet/TestData/outputTest/6/user26_fluorescent_0.npz] saved.
[/home/zhang/S2/RP/DataSet/TestData/outputTest/7/user26_fluorescent_0.npz] saved.
[/home/zhang/S2/RP/DataSet/TestData/outputTest/7/user26_fluorescent_1.npz] saved.
[/home/zhang/S2/RP/DataSet/TestData/outputTest/8/user26_fluorescent_0.npz] saved.
[/home/zhang/S2/RP/DataSet/TestData/outputTest/9/user26_fluorescent_0.npz] saved.
[/home/zh

In [5]:
def classify_aedat_to_npz(origin_dataset_path: str, events_npz_path: str):

    train_dir = os.path.join(events_npz_path, 'train')
    test_dir = os.path.join(events_npz_path, 'test')
    max_threads_number_for_datasets_preprocess = 16  #my computer has 16 threads 
    os.mkdir(train_dir)
    os.mkdir(test_dir)
    print(f'Mkdir [{train_dir, test_dir}].')
    for label in range(11):
        os.mkdir(os.path.join(train_dir, str(label)))
        os.mkdir(os.path.join(test_dir, str(label)))
    print(f'Mkdir {os.listdir(train_dir)} in [{train_dir}] and {os.listdir(test_dir)} in [{test_dir}].')

    with open(os.path.join(origin_dataset_path, 'trials_to_train.txt')) as trials_to_train_txt, open(
            os.path.join(origin_dataset_path, 'trials_to_test.txt')) as trials_to_test_txt:
        # use multi-thread to accelerate
        t_ckp = time.time()
        with ThreadPoolExecutor(max_workers=min(multiprocessing.cpu_count(), max_threads_number_for_datasets_preprocess)) as tpe:
            print(f'Start the ThreadPoolExecutor with max workers = [{tpe._max_workers}].')

            for fname in trials_to_train_txt.readlines():
                fname = fname.strip()
                if fname.__len__() > 0:  #we can see in the files some lines are empty
                    aedat_file = os.path.join(origin_dataset_path, fname)
                    fname = os.path.splitext(fname)[0] #remove .aedat
                    tpe.submit(split_aedat_files_to_npz, fname, aedat_file, os.path.join(origin_dataset_path, fname + '_labels.csv'), train_dir)

            for fname in trials_to_test_txt.readlines():
                fname = fname.strip()
                if fname.__len__() > 0:
                    aedat_file = os.path.join(origin_dataset_path, fname)
                    fname = os.path.splitext(fname)[0]  #remove .aedat
                    tpe.submit(split_aedat_files_to_npz, fname, aedat_file,
                                os.path.join(origin_dataset_path, fname + '_labels.csv'), test_dir)

        print(f'Used time = [{round(time.time() - t_ckp, 2)}s].')
    print(f'All aedat files have been split to samples and saved into [{train_dir, test_dir}].')

In [9]:
classify_aedat_to_npz(dvs_path,events_dvs_split_path)

Mkdir [('/home/zhang/S2/RP/DataSet/DVS_npz_events/train', '/home/zhang/S2/RP/DataSet/DVS_npz_events/test')].
Mkdir ['2', '5', '4', '10', '9', '1', '7', '3', '6', '0', '8'] in [/home/zhang/S2/RP/DataSet/DVS_npz_events/train] and ['2', '5', '4', '10', '9', '1', '7', '3', '6', '0', '8'] in [/home/zhang/S2/RP/DataSet/DVS_npz_events/test].
Start the ThreadPoolExecutor with max workers = [16].
Start to split [/home/zhang/S2/RP/DataSet/DVS_Gesture_dataset/DvsGesture/user04_fluorescent_led.aedat] to samples.
[/home/zhang/S2/RP/DataSet/DVS_npz_events/train/0/user04_fluorescent_led_0.npz] saved.
[/home/zhang/S2/RP/DataSet/DVS_npz_events/train/1/user04_fluorescent_led_0.npz] saved.
[/home/zhang/S2/RP/DataSet/DVS_npz_events/train/2/user04_fluorescent_led_0.npz] saved.
[/home/zhang/S2/RP/DataSet/DVS_npz_events/train/3/user04_fluorescent_led_0.npz] saved.
[/home/zhang/S2/RP/DataSet/DVS_npz_events/train/4/user04_fluorescent_led_0.npz] saved.
Start to split [/home/zhang/S2/RP/DataSet/DVS_Gesture_datas

In [8]:
#Check file classified
r = np.load(os.path.join(events_dvs_split_path,"test/9/user29_lab_0.npz"))
for i in r.keys():
    print(i,r[i])

t [180122762 180122794 180122807 ... 188583498 188583504 188583508]
x [52 91 45 ... 45 51 54]
y [ 85  79  90 ... 108 113 115]
p [1 0 0 ... 1 1 1]


In [6]:
def create_data_set_events(npz_dir: str):
    train_data = []
    test_data = []
    train_label = []
    test_label = []

    npz_dir_list = os.listdir(npz_dir) #dir train,test
    for dir0 in npz_dir_list:
        dir1 = os.path.join(npz_dir,dir0) 
        classes_dir = os.listdir(dir1) #dir 0,1,2...
        for dir2 in classes_dir:
            file_dir = os.path.join(dir1,dir2) 
            filrs_names = os.listdir(file_dir)
            for file_name in filrs_names:
                npz_file = np.load(os.path.join(file_dir,file_name))
                t=npz_file['t']
                x=npz_file['x']
                y=npz_file['y']
                p=npz_file['p']
                txyp = {'t': t,'x': x,'y': y,'p': p}
                if dir0 =="train":
                    train_data.append(txyp)
                    train_label.append(int(dir2)) 
                else:
                    test_data.append(txyp)
                    test_label.append(int(dir2))
                    
    return np.array(train_data), np.array(train_label), np.array(test_data), np.array(test_label)

In [7]:
train_data,train_label,test_data,test_label = create_data_set_events(events_dvs_split_path)

In [9]:
num_train = len(train_data)
num_test = len(test_data)
print(f'We have {num_train} samples in the training set')
print(f'We have {num_test} samples in the test set')
print(f'Total samples: {num_test+num_train}')

We have 1176 samples in the training set
We have 288 samples in the test set
Total samples: 1464


In [12]:
#Check dataset split
events,label = train_data[25],train_label[25]
for k in events.keys():
    print(k,events[k])
print('label',label)

t [66802955 66802957 66802972 ... 71631530 71631566 71631587]
x [91 83 85 ... 83 85 81]
y [ 66 102  69 ...  86  82  71]
p [1 1 1 ... 1 1 1]
label 2


-----
-----
## Convert events stream to frames stream

In [8]:
def create_same_directory_structure(source_dir: str, target_dir: str):
    for sub_dir_name in os.listdir(source_dir):
        source_sub_dir = os.path.join(source_dir, sub_dir_name)
        if os.path.isdir(source_sub_dir):
            target_sub_dir = os.path.join(target_dir, sub_dir_name)
            os.mkdir(target_sub_dir)
            print(f'Mkdir [{target_sub_dir}].')
            create_same_directory_structure(source_sub_dir, target_sub_dir)

In [22]:
#Create same structure for frames like events above for each type of spliting
create_same_directory_structure(events_dvs_split_path, "/home/zhang/S2/RP/DataSet/DVS_npz_frames/split_by_number/")
create_same_directory_structure(events_dvs_split_path, "/home/zhang/S2/RP/DataSet/DVS_npz_frames/split_by_time/")

Mkdir [/home/zhang/S2/RP/DataSet/DVS_npz_frames/Split_by_number/test].
Mkdir [/home/zhang/S2/RP/DataSet/DVS_npz_frames/Split_by_number/test/2].
Mkdir [/home/zhang/S2/RP/DataSet/DVS_npz_frames/Split_by_number/test/5].
Mkdir [/home/zhang/S2/RP/DataSet/DVS_npz_frames/Split_by_number/test/4].
Mkdir [/home/zhang/S2/RP/DataSet/DVS_npz_frames/Split_by_number/test/10].
Mkdir [/home/zhang/S2/RP/DataSet/DVS_npz_frames/Split_by_number/test/9].
Mkdir [/home/zhang/S2/RP/DataSet/DVS_npz_frames/Split_by_number/test/1].
Mkdir [/home/zhang/S2/RP/DataSet/DVS_npz_frames/Split_by_number/test/7].
Mkdir [/home/zhang/S2/RP/DataSet/DVS_npz_frames/Split_by_number/test/3].
Mkdir [/home/zhang/S2/RP/DataSet/DVS_npz_frames/Split_by_number/test/6].
Mkdir [/home/zhang/S2/RP/DataSet/DVS_npz_frames/Split_by_number/test/0].
Mkdir [/home/zhang/S2/RP/DataSet/DVS_npz_frames/Split_by_number/test/8].
Mkdir [/home/zhang/S2/RP/DataSet/DVS_npz_frames/Split_by_number/train].
Mkdir [/home/zhang/S2/RP/DataSet/DVS_npz_frames/Split

In [20]:
create_same_directory_structure(events_dvs_split_path, "/home/zhang/S2/RP/DataSet/DVS_npz_frames/split_by_fixed_duration/")

Mkdir [/home/zhang/S2/RP/DataSet/DVS_npz_frames/split_by_fixed_duration/test].
Mkdir [/home/zhang/S2/RP/DataSet/DVS_npz_frames/split_by_fixed_duration/test/2].
Mkdir [/home/zhang/S2/RP/DataSet/DVS_npz_frames/split_by_fixed_duration/test/5].
Mkdir [/home/zhang/S2/RP/DataSet/DVS_npz_frames/split_by_fixed_duration/test/4].
Mkdir [/home/zhang/S2/RP/DataSet/DVS_npz_frames/split_by_fixed_duration/test/10].
Mkdir [/home/zhang/S2/RP/DataSet/DVS_npz_frames/split_by_fixed_duration/test/9].
Mkdir [/home/zhang/S2/RP/DataSet/DVS_npz_frames/split_by_fixed_duration/test/1].
Mkdir [/home/zhang/S2/RP/DataSet/DVS_npz_frames/split_by_fixed_duration/test/7].
Mkdir [/home/zhang/S2/RP/DataSet/DVS_npz_frames/split_by_fixed_duration/test/3].
Mkdir [/home/zhang/S2/RP/DataSet/DVS_npz_frames/split_by_fixed_duration/test/6].
Mkdir [/home/zhang/S2/RP/DataSet/DVS_npz_frames/split_by_fixed_duration/test/0].
Mkdir [/home/zhang/S2/RP/DataSet/DVS_npz_frames/split_by_fixed_duration/test/8].
Mkdir [/home/zhang/S2/RP/Data

Denote ``frames_num`` as `T`, if ``split_by`` is ``'time'``, then
$$\Delta T = [\frac{t_{N-1} - t_{0}}{T}]$$
$$j_{l} = \mathop{\arg\min}\limits_{k} {t_{k} | t_{k} \geq t_{0} + \Delta T \cdot j}$$
$$j_{r} = \begin{cases} \mathop{\arg\max}\limits_{k} \{t_{k} | t_{k} < t_{0} + \Delta T \cdot (j + 1)\} + 1, & j <  T - 1 \cr N, & j = T - 1 \end{cases}$$
If ``split_by`` is ``'number'``, then

$$j_{l} = [\frac{N}{T}] \cdot j $$
$$j_{r} = \begin{cases} [\frac{N}{T}] \cdot (j + 1), & j <  T - 1 \cr N, & j = T - 1 \end{cases}$$

Where `T` is number of frames split, `j` is index of frame, `N` is number of events

Denote a two channels frame as :

A frame in the frame data after integration noted `F(j)` and a pixel at $(p, x, y)$ as $F(j, p, x, y)$, the pixel value is integrated from the events data whose indices are in $[j_{l}, j_{r})$:

$$
\begin{aligned}
j_l & =\left\lfloor\frac{N}{T}\right\rfloor \cdot j \\
j_r & = \begin{cases}\left\lfloor\frac{N}{T}\right\rfloor \cdot(j+1), & \text { if } j<T-1 \\
N, & \text { if } j=T-1\end{cases} \\
F(j, p, x, y) & =\sum_{i=j_l}^{j_r-1} \mathcal{I}_{p, x, y}\left(p_i, x_i, y_i\right)
\end{aligned}
$$
where $\lfloor$ $\cdot$ $\rfloor$ is the floor operation, T is number of frames split.
$$\mathcal{I}_{p, x, y}(p_{i}, x_{i}, y_{i})$$ 
is an indicator function and it equals 1 only when $$(p, x, y) = (p_{i}, x_{i}, y_{i})$$


In [9]:
def integrate_events_segment_to_frame(x: np.ndarray, y: np.ndarray, p: np.ndarray, H: int, W: int, j_l: int, j_r: int) -> np.ndarray:
    frame = np.zeros(shape=[2, H * W])
    x = x[j_l: j_r].astype(int)  # avoid overflow
    y = y[j_l: j_r].astype(int)
    p = p[j_l: j_r]
    mask = []
    mask.append(p == 0)
    mask.append(np.logical_not(mask[0]))
    for c in range(2):
        position = y[mask[c]] * W + x[mask[c]]
        events_number_per_pos = np.bincount(position)
        frame[c][np.arange(events_number_per_pos.size)] += events_number_per_pos
    return frame.reshape((2, H, W))

In [10]:
def get_fixed_frames_number_segment_index(events_t: np.ndarray, split_by: str, frames_num: int) -> tuple:
    j_l = np.zeros(shape=[frames_num], dtype=int)
    j_r = np.zeros(shape=[frames_num], dtype=int)
    N = events_t.size

    if split_by == 'number':
        di = N // frames_num
        for i in range(frames_num):
            j_l[i] = i * di
            j_r[i] = j_l[i] + di
        j_r[-1] = N

    elif split_by == 'time':
        dt = (events_t[-1] - events_t[0]) // frames_num
        idx = np.arange(N)
        for i in range(frames_num):
            t_l = dt * i + events_t[0]
            t_r = t_l + dt
            mask = np.logical_and(events_t >= t_l, events_t < t_r)
            idx_masked = idx[mask]
            j_l[i] = idx_masked[0]
            j_r[i] = idx_masked[-1] + 1

        j_r[-1] = N
    else:
        raise NotImplementedError

    return j_l, j_r

In [11]:
def integrate_events_by_fixed_frames_number(events: Dict, label: str, split_by: str, frames_num: int, H: int, W: int) -> np.ndarray:
    '''
    :param events: a dict whose keys are ``['t', 'x', 'y', 'p']`` and values are ``numpy.ndarray``
    :param split_by: 'time' or 'number'
    :param frames_num: the number of frames
    :param H: the height of frame
    :param W: the weight of frame
    :return: frames,label
    Integrate events to frames by fixed frames number.
    '''
    t, x, y, p = (events[key] for key in ('t', 'x', 'y', 'p'))
    j_l, j_r = get_fixed_frames_number_segment_index(t, split_by, frames_num)
    frames = np.zeros([frames_num, 2, H, W])
    for i in range(frames_num):
        frames[i] = integrate_events_segment_to_frame(x, y, p, H, W, j_l[i], j_r[i])
    return frames,label

In [14]:
#Test function
events,label = train_data[500],train_label[500]
frames,label = integrate_events_by_fixed_frames_number(events=events,label=label, split_by='number',frames_num=20,H=128,W=128)
print(frames.shape)
print(label)

(20, 2, 128, 128)
1


In [12]:
def integrate_events_by_fixed_duration(events: Dict, duration: int, H: int, W: int) -> np.ndarray:
    '''
    Integrate events to frames by fixed time duration of each frame.
    '''
    x = events['x']
    y = events['y']
    t = events['t']
    p = events['p']
    N = t.size

    frames = []
    left = 0
    right = 0
    while True:
        t_l = t[left]
        while True:
            if right == N or t[right] - t_l > duration:
                break
            else:
                right += 1
        # integrate from index [left, right)
        frames.append(np.expand_dims(integrate_events_segment_to_frame(x, y, p, H, W, left, right), 0))

        left = right

        if right == N:
            return np.concatenate(frames)

In [13]:
def padding_zeros_with_variable_length(X):
    x_list = []
    x_len_list = []
    for x  in X:
        x_list.append(torch.as_tensor(x))
        x_len_list.append(x.shape[0])

    return torch.nn.utils.rnn.pad_sequence(x_list, batch_first=True).numpy(), np.array(x_len_list)

In [14]:
def save_frame_to_gif(frames: torch.Tensor or np.ndarray, label:str, save_path: str,unique_ = "") -> None:
    '''
    frames: frames with `shape=[T, 2, H, W]`
         save frames to a gif file in the directory `save_path`
    '''
    if isinstance(frames, np.ndarray):
        frames = torch.from_numpy(frames)
    to_img = transforms.ToPILImage()
    img_tensor = torch.zeros([frames.shape[0], 3, frames.shape[2], frames.shape[3]])
    img_tensor[:, 1] = frames[:, 0]
    img_tensor[:, 2] = frames[:, 1]

    gif_frames = []
    for t in range(img_tensor.shape[0]):
        img = to_img(img_tensor[t])
        plt.figure(figsize=(15, 15))
        plt.imshow(img)
        plt.title(f"Visualization frame_{t}")
        plt.savefig(os.path.join(save_path,f"frame_{t}.png"))
        plt.close()
        img_read = imageio.v2.imread(os.path.join(save_path,f"frame_{t}.png"))
        gif_frames.append(img_read)

    imageio.mimsave(os.path.join(save_path,f"visual_label_{label}"+f"{unique_}.gif"), # output gif
            gif_frames,          # array of input frames
            fps = 10)         # frames per second
    
    for file in os.listdir(save_path):  #remove png generated
        if(file[:5]=="frame"):
            os.remove(os.path.join(save_path,file))

    print(f'Save gif to [{save_path+f"/visual_label_{label}"+f"{unique_}.gif"}].')

In [30]:
#For each class we save a gif example
class_diff_index_list = [0,98,196,294,392,490,686,784,883,1000,1078]
for i in class_diff_index_list:
    events_,label_ = train_data[i],train_label[i]
    frames_,label_ = integrate_events_by_fixed_frames_number(events=events_,label=label_, split_by='number',frames_num=20,H=128,W=128)
    save_frame_to_gif(frames_,label_,visualization_saved_path,unique_="_num_2222")

Save gif to [/home/zhang/S2/RP/DataSet/Visualization_saved/visualization_label_2_num_2222.gif].
Save gif to [/home/zhang/S2/RP/DataSet/Visualization_saved/visualization_label_5_num_2222.gif].
Save gif to [/home/zhang/S2/RP/DataSet/Visualization_saved/visualization_label_4_num_2222.gif].
Save gif to [/home/zhang/S2/RP/DataSet/Visualization_saved/visualization_label_10_num_2222.gif].
Save gif to [/home/zhang/S2/RP/DataSet/Visualization_saved/visualization_label_9_num_2222.gif].
Save gif to [/home/zhang/S2/RP/DataSet/Visualization_saved/visualization_label_1_num_2222.gif].
Save gif to [/home/zhang/S2/RP/DataSet/Visualization_saved/visualization_label_7_num_2222.gif].
Save gif to [/home/zhang/S2/RP/DataSet/Visualization_saved/visualization_label_3_num_2222.gif].
Save gif to [/home/zhang/S2/RP/DataSet/Visualization_saved/visualization_label_6_num_2222.gif].
Save gif to [/home/zhang/S2/RP/DataSet/Visualization_saved/visualization_label_0_num_2222.gif].
Save gif to [/home/zhang/S2/RP/DataSet/

In [21]:
create_same_directory_structure(frames_dvs_split_path,visualization_saved_path)

Mkdir [/home/zhang/S2/RP/DataSet/Visualization_saved/split_by_time].
Mkdir [/home/zhang/S2/RP/DataSet/Visualization_saved/split_by_time/test].
Mkdir [/home/zhang/S2/RP/DataSet/Visualization_saved/split_by_time/test/2].
Mkdir [/home/zhang/S2/RP/DataSet/Visualization_saved/split_by_time/test/5].
Mkdir [/home/zhang/S2/RP/DataSet/Visualization_saved/split_by_time/test/4].
Mkdir [/home/zhang/S2/RP/DataSet/Visualization_saved/split_by_time/test/10].
Mkdir [/home/zhang/S2/RP/DataSet/Visualization_saved/split_by_time/test/9].
Mkdir [/home/zhang/S2/RP/DataSet/Visualization_saved/split_by_time/test/1].
Mkdir [/home/zhang/S2/RP/DataSet/Visualization_saved/split_by_time/test/7].
Mkdir [/home/zhang/S2/RP/DataSet/Visualization_saved/split_by_time/test/3].
Mkdir [/home/zhang/S2/RP/DataSet/Visualization_saved/split_by_time/test/6].
Mkdir [/home/zhang/S2/RP/DataSet/Visualization_saved/split_by_time/test/0].
Mkdir [/home/zhang/S2/RP/DataSet/Visualization_saved/split_by_time/test/8].
Mkdir [/home/zhang/S

In [15]:
def save_events_data_to_frame_to_dir(events_data: np.ndarray, events_data_label: np.ndarray,data_type:str, npz_save_path_root: str, split_by: str, frames_num: int, H: int, W: int, visual_save_path_root =None):
    num_data = len(events_data)
    for i in range(num_data):
        event_,label_ = events_data[i],events_data_label[i]
        frames_, label_ = integrate_events_by_fixed_frames_number(events=event_,label=label_,split_by=split_by,frames_num=frames_num,H=H,W=W)
        if visual_save_path_root is not None:
            save_frame_to_gif(frames_,label_,os.path.join(visual_save_path_root,data_type,str(label_)),unique_="_num_"+str(i))
        np.save(os.path.join(npz_save_path_root,data_type,str(label_),str(i)),frames_)
        print(f"save npy to [{os.path.join(npz_save_path_root,data_type,str(label_),str(i))}.npy]")

In [16]:
def save_events_data_to_frame_npz_by_fixed_frames_number(events_train_data: np.ndarray, events_train_label: np.ndarray, events_test_data: np.ndarray, events_test_label : np.ndarray, npz_save_path_root: str, split_by: str, frames_num: int, H: int, W: int, visual_save_path =None)-> None:
    assert frames_num > 0 and isinstance(frames_num, int)
    assert split_by == 'time' or split_by == 'number' or split_by == "fixed_duration"
    if split_by=="number":
        npz_root_path = os.path.join(npz_save_path_root,"split_by_number")
    elif split_by=="time":
        npz_root_path = os.path.join(npz_save_path_root,"split_by_time")

    if visual_save_path is not None:
        if split_by=="number":
            visual_save_path = os.path.join(visual_save_path,"split_by_number")
        elif split_by=="time":
            visual_save_path = os.path.join(visual_save_path,"split_by_time")

    t_ckp = time.time()
    with ThreadPoolExecutor(max_workers=min(multiprocessing.cpu_count(), 16)) as tpe:
        print(f'Start the ThreadPoolExecutor with max workers = [{tpe._max_workers}].')
        tpe.submit(save_events_data_to_frame_to_dir,events_train_data,events_train_label,"train",npz_root_path,split_by,frames_num,H,W,visual_save_path)
        tpe.submit(save_events_data_to_frame_to_dir,events_test_data,events_test_label,"test",npz_root_path,split_by,frames_num,H,W,visual_save_path)
    #save_events_data_to_frame_to_dir(events_train_data,events_train_label,"train",npz_root_path,split_by,frames_num,H,W,visual_save_path)
    #save_events_data_to_frame_to_dir(events_test_data,events_test_label,"test",npz_root_path,split_by,frames_num,H,W,visual_save_path)

    print(f'Used time = [{round(time.time() - t_ckp, 2)}s].')
    print(f'All files have been saved.')
        
    

In [15]:
save_events_data_to_frame_npz_by_fixed_frames_number(train_data,train_label,test_data,test_label,frames_dvs_split_path,"number",20,128,128,visual_save_path = None)

Start the ThreadPoolExecutor with max workers = [16].
save npy to [/home/zhang/S2/RP/DataSet/DVS_npz_frames/split_by_number/test/2/0.npy]
save npy to [/home/zhang/S2/RP/DataSet/DVS_npz_frames/split_by_number/train/2/0.npy]
save npy to [/home/zhang/S2/RP/DataSet/DVS_npz_frames/split_by_number/test/2/1.npy]
save npy to [/home/zhang/S2/RP/DataSet/DVS_npz_frames/split_by_number/train/2/1.npy]
save npy to [/home/zhang/S2/RP/DataSet/DVS_npz_frames/split_by_number/train/2/2.npy]
save npy to [/home/zhang/S2/RP/DataSet/DVS_npz_frames/split_by_number/test/2/2.npy]
save npy to [/home/zhang/S2/RP/DataSet/DVS_npz_frames/split_by_number/train/2/3.npy]
save npy to [/home/zhang/S2/RP/DataSet/DVS_npz_frames/split_by_number/train/2/4.npy]
save npy to [/home/zhang/S2/RP/DataSet/DVS_npz_frames/split_by_number/test/2/3.npy]
save npy to [/home/zhang/S2/RP/DataSet/DVS_npz_frames/split_by_number/train/2/5.npy]
save npy to [/home/zhang/S2/RP/DataSet/DVS_npz_frames/split_by_number/test/2/4.npy]
save npy to [/ho

In [16]:
save_events_data_to_frame_npz_by_fixed_frames_number(train_data,train_label,test_data,test_label,frames_dvs_split_path,"time",20,128,128,visual_save_path = None)

Start the ThreadPoolExecutor with max workers = [16].
save npy to [/home/zhang/S2/RP/DataSet/DVS_npz_frames/split_by_time/test/2/0.npy]
save npy to [/home/zhang/S2/RP/DataSet/DVS_npz_frames/split_by_time/train/2/0.npy]
save npy to [/home/zhang/S2/RP/DataSet/DVS_npz_frames/split_by_time/test/2/1.npy]
save npy to [/home/zhang/S2/RP/DataSet/DVS_npz_frames/split_by_time/train/2/1.npy]
save npy to [/home/zhang/S2/RP/DataSet/DVS_npz_frames/split_by_time/train/2/2.npy]
save npy to [/home/zhang/S2/RP/DataSet/DVS_npz_frames/split_by_time/test/2/2.npy]
save npy to [/home/zhang/S2/RP/DataSet/DVS_npz_frames/split_by_time/train/2/3.npy]
save npy to [/home/zhang/S2/RP/DataSet/DVS_npz_frames/split_by_time/test/2/3.npy]
save npy to [/home/zhang/S2/RP/DataSet/DVS_npz_frames/split_by_time/train/2/4.npy]
save npy to [/home/zhang/S2/RP/DataSet/DVS_npz_frames/split_by_time/test/2/4.npy]
save npy to [/home/zhang/S2/RP/DataSet/DVS_npz_frames/split_by_time/train/2/5.npy]
save npy to [/home/zhang/S2/RP/DataSet

In [17]:
def save_events_data_to_frame_npy_by_fixed_time_duration(events_train_data: np.ndarray, events_train_label: np.ndarray, events_test_data: np.ndarray, events_test_label : np.ndarray, npy_save_path_root: str,durantion:int, H: int, W: int, visual_save_path =None)-> np.ndarray:
    len_train = len(events_train_data)
    len_test = len(events_test_data)
    data = np.concatenate((events_train_data,events_test_data))
    data_label = np.concatenate((events_train_label,events_test_label))
    train_save_path = os.path.join(npy_save_path_root,"split_by_fixed_duration","train")
    test_save_path = os.path.join(npy_save_path_root,"split_by_fixed_duration","test")
    frames_variable_len_list = []

    for e in data:
        frames_variable_len_list.append(integrate_events_by_fixed_duration(e,durantion,H,W)) 
    frames_padding,frames_origin_len_list = padding_zeros_with_variable_length(frames_variable_len_list)
    t_ckp = time.time()

    for i in range(len_train):
        np.save(os.path.join(train_save_path,str(data_label[i]),str(i)),frames_padding[i])
        print(f"save npy to [{os.path.join(train_save_path,str(data_label[i]),str(i))}.npy]") 
        if visual_save_path is not None:
            save_frame_to_gif(frames_padding[i],data_label[i],os.path.join(visual_save_path,"split_by_fixed_duration","train",str(data_label[i])),unique_="_num_"+str(i))
    for i in range(len_train,len_train+len_test):
        np.save(os.path.join(test_save_path,str(data_label[i]),str(i)),frames_padding[i])
        print(f"save npy to [{os.path.join(test_save_path,str(data_label[i]),str(i))}.npy]")
        if visual_save_path is not None:
            save_frame_to_gif(frames_padding[i],data_label[i],os.path.join(visual_save_path,"split_by_fixed_duration","test",str(data_label[i])),unique_="_num_"+str(i))

    print(f'Used time = [{round(time.time() - t_ckp, 2)}s].')
    print(f'All files have been saved.')
    
    return frames_origin_len_list

In [19]:
frames_origin_len_list = save_events_data_to_frame_npy_by_fixed_time_duration(train_data,train_label,test_data,test_label,frames_dvs_split_path,500000,128,128)

save npy to [/home/zhang/S2/RP/DataSet/DVS_npz_frames/split_by_fixed_duration/train/2/0.npy]
save npy to [/home/zhang/S2/RP/DataSet/DVS_npz_frames/split_by_fixed_duration/train/2/1.npy]
save npy to [/home/zhang/S2/RP/DataSet/DVS_npz_frames/split_by_fixed_duration/train/2/2.npy]
save npy to [/home/zhang/S2/RP/DataSet/DVS_npz_frames/split_by_fixed_duration/train/2/3.npy]
save npy to [/home/zhang/S2/RP/DataSet/DVS_npz_frames/split_by_fixed_duration/train/2/4.npy]
save npy to [/home/zhang/S2/RP/DataSet/DVS_npz_frames/split_by_fixed_duration/train/2/5.npy]
save npy to [/home/zhang/S2/RP/DataSet/DVS_npz_frames/split_by_fixed_duration/train/2/6.npy]
save npy to [/home/zhang/S2/RP/DataSet/DVS_npz_frames/split_by_fixed_duration/train/2/7.npy]
save npy to [/home/zhang/S2/RP/DataSet/DVS_npz_frames/split_by_fixed_duration/train/2/8.npy]
save npy to [/home/zhang/S2/RP/DataSet/DVS_npz_frames/split_by_fixed_duration/train/2/9.npy]
save npy to [/home/zhang/S2/RP/DataSet/DVS_npz_frames/split_by_fixed_d

In [18]:
def create_data_set_frames(npy_dir: str):
    train_data = []
    test_data = []
    train_label = []
    test_label = []

    npz_dir_list = os.listdir(npy_dir) #dir train,test
    for dir0 in npz_dir_list:
        dir1 = os.path.join(npy_dir,dir0) 
        classes_dir = os.listdir(dir1) #dir 0,1,2...
        for dir2 in classes_dir:
            file_dir = os.path.join(dir1,dir2) 
            filrs_names = os.listdir(file_dir)
            for file_name in filrs_names:
                npy = np.load(os.path.join(file_dir,file_name))
                if dir0 =="train":
                    train_data.append(npy)
                    train_label.append(int(dir2)) 
                else:
                    test_data.append(npy)
                    test_label.append(int(dir2))
                    
    return np.array(train_data), np.array(train_label), np.array(test_data), np.array(test_label)

In [20]:
train_data_frames, train_label_frames, test_data_frames, test_label_frames = create_data_set_frames(os.path.join(frames_dvs_split_path,"split_by_fixed_duration"))
save_path=os.path.join(frames_dvs_split_path,"split_by_fixed_duration")
np.save(os.path.join(save_path,"train_data_frames_fixed_duration"),np.float32(train_data_frames))
np.save(os.path.join(save_path,"train_label_frames_fixed_duration"),np.float32(train_label_frames))
np.save(os.path.join(save_path,"test_data_frames_fixed_duration"),np.float32(test_data_frames))
np.save(os.path.join(save_path,"test_label_frames_fixed_duration"),np.float32(test_label_frames))

train_data_frames, train_label_frames, test_data_frames, test_label_frames = create_data_set_frames(os.path.join(frames_dvs_split_path,"split_by_number"))
save_path=os.path.join(frames_dvs_split_path,"split_by_number")
np.save(os.path.join(save_path,"train_data_frames_number"),np.float32(train_data_frames))
np.save(os.path.join(save_path,"train_label_frames_number"),np.float32(train_label_frames))
np.save(os.path.join(save_path,"test_data_frames_number"),np.float32(test_data_frames))
np.save(os.path.join(save_path,"test_label_frames_number"),np.float32(test_label_frames))

train_data_frames, train_label_frames, test_data_frames, test_label_frames = create_data_set_frames(os.path.join(frames_dvs_split_path,"split_by_time"))
save_path=os.path.join(frames_dvs_split_path,"split_by_time")
np.save(os.path.join(save_path,"train_data_frames_time"),np.float32(train_data_frames))
np.save(os.path.join(save_path,"train_label_frames_time"),np.float32(train_label_frames))
np.save(os.path.join(save_path,"test_data_frames_time"),np.float32(test_data_frames))
np.save(os.path.join(save_path,"test_label_frames_time"),np.float32(test_label_frames))


In [5]:
X_train = np.load("/home/zhang/S2/RP/DataSet/DVS_npz_frames/split_by_fixed_duration/train_data_frames_fixed_duration.npy")
y_train = np.load("/home/zhang/S2/RP/DataSet/DVS_npz_frames/split_by_fixed_duration/train_label_frames_fixed_duration.npy")

X_test = np.load("/home/zhang/S2/RP/DataSet/DVS_npz_frames/split_by_fixed_duration/test_data_frames_fixed_duration.npy")
y_test = np.load("/home/zhang/S2/RP/DataSet/DVS_npz_frames/split_by_fixed_duration/test_label_frames_fixed_duration.npy")

In [ ]:
print(X_train.shape)
print(X_test.shape)
print(y_train.shape)

In [21]:
X_test = np.load("/home/zhang/S2/RP/DataSet/DVS_npz_frames/split_by_fixed_duration/test_data_frames_fixed_duration.npy")
print(X_test.dtype)

float32


In [22]:
X_f = X_test.flatten()
print(X_f[173])

1.0
